In [128]:
!pip install requests
!pip install numpy pandas
!pip install python-dotenv


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


### Function to calculate road distance b/w any 2 loactions

In [129]:
import numpy as np
import pandas as pd
import time

In [130]:
from dotenv import load_dotenv
import os
load_dotenv()
openrouteservice_api_key=os.getenv("openrouteservice_api_key")

In [131]:
import requests

def get_road_distance_and_time(start_coords, end_coords, profile='driving-car'):
    api_key=openrouteservice_api_key
    base_url = 'https://api.openrouteservice.org/v2/directions/'
    url = f'{base_url}{profile}'
    params = {
        'api_key': api_key,
        'start': f'{start_coords[1]},{start_coords[0]}',  #order: longitude,latitude
        'end': f'{end_coords[1]},{end_coords[0]}',
    }
    try:
        response = requests.get(url, params=params)
        data = response.json()
        print(data)
        if 'features' in data and len(data['features']) > 0:
            distance = data['features'][0]['properties']['segments'][0]['distance']
            timebwNodes = data['features'][0]['properties']['segments'][0]['duration']
            return [distance,timebwNodes]
        else:
            print('No route found.')
            return None
    except requests.exceptions.RequestException as e:
        print(f'Error: {e}')
        return None

In [132]:
# dist = get_road_distance((12.9929356,77.5951078),(12.9694440,77.7499922))
# print(dist)

### Fetch Data

In [133]:
import requests

def get_coordinates(place_name):
    base_url = 'https://nominatim.openstreetmap.org/search'
    params = {
        'q': place_name,
        'format': 'json',
        'limit': 1,
    }
    response = requests.get(base_url, params=params)
    data = response.json()
    # print(data)
    if data:
        location = (float(data[0]['lat']), float(data[0]['lon']))
        return location
    else:
        return None

charging_stations = [get_coordinates('charging_station, Bangalore Indira Nagar'),get_coordinates('charging_station, Bangalore Electronic city')]

bus_stop_coordinates = []
ind = 0
while len(bus_stop_coordinates) < 13:
    bus_stop = get_coordinates(f'Bus Stop {ind}, Bangalore')
    ind=ind+1
    if bus_stop:
        bus_stop_coordinates.append(bus_stop)
    bus_stop_coordinates=list(set(bus_stop_coordinates))

car_workshop = get_coordinates('Car Workshop, Bangalore')

In [134]:
bus_stop_coordinates

[(13.1067756, 77.6005477),
 (13.06002, 77.5766226),
 (13.0887623, 77.5823852),
 (13.0985096, 77.6273422),
 (13.0686238, 77.4765245),
 (13.0499117, 77.516421),
 (13.0757683, 77.4726196),
 (13.0764541, 77.4585122),
 (13.0562923, 77.6328481),
 (13.1180494, 77.5777231),
 (12.9774995, 77.5900342),
 (13.1144798, 77.5677789),
 (12.9767773, 77.5732599)]

In [135]:
car_workshop

(12.9443978, 77.6514662)

In [136]:
charging_stations

[(12.975668, 77.6413089), (12.9154635, 77.6158909)]

### Sample Data
- car_workshop : list of depot (0) 
- chargind_stations : BSS 
- bust_stop_coordinates : customers 

### Sample Space
- [depot, ...BSS, ...customers, depot]

In [137]:
nodes = [car_workshop]+ charging_stations + bus_stop_coordinates + [car_workshop]

### distance matrix

In [138]:

number_of_node = len(nodes)
distance_matrix = np.zeros((number_of_node, number_of_node))
distance_matrix = pd.DataFrame(distance_matrix)
time_matrix = np.zeros((number_of_node, number_of_node))
time_matrix = pd.DataFrame(time_matrix)

for i in range(number_of_node):
    for j in range(number_of_node):
            distTime=get_road_distance_and_time(nodes[i],nodes[j])
            distance_matrix.iloc[i, j] = distTime[0]/1000
            time_matrix.iloc[i,j]=distTime[1]/3600
            time.sleep(2)


{'type': 'FeatureCollection', 'metadata': {'attribution': 'openrouteservice.org | OpenStreetMap contributors', 'service': 'routing', 'timestamp': 1694780717232, 'query': {'coordinates': [[77.6514662, 12.9443978], [77.6514662, 12.9443978]], 'profile': 'driving-car', 'format': 'json'}, 'engine': {'version': '7.1.0', 'build_date': '2023-07-09T01:31:50Z', 'graph_date': '2023-09-03T10:10:37Z'}}, 'bbox': [77.651477, 12.944438, 77.651477, 12.944438], 'features': [{'bbox': [77.651477, 12.944438, 77.651477, 12.944438], 'type': 'Feature', 'properties': {'transfers': 0, 'fare': 0, 'segments': [{'distance': 0.0, 'duration': 0.0, 'steps': [{'distance': 0.0, 'duration': 0.0, 'type': 11, 'instruction': 'Head east', 'name': '-', 'way_points': [0, 0]}]}], 'way_points': [0, 0], 'summary': {}}, 'geometry': {'coordinates': [[77.651477, 12.944438]], 'type': 'LineString'}}]}
{'type': 'FeatureCollection', 'metadata': {'attribution': 'openrouteservice.org | OpenStreetMap contributors', 'service': 'routing', '

In [139]:
distance_matrix

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0.0000,6.2851,8.9089,24.8315,19.7986,26.8570,23.0420,30.0764,23.8108,31.0103,32.8257,17.6433,28.0415,11.2359,28.4530,12.7243,0.0000
1,4.4247,0.0000,9.4583,20.1860,15.1531,22.2115,18.3965,25.7195,19.4539,26.6533,28.4687,12.9977,23.3959,7.4903,23.8075,10.0275,4.4247
2,10.3362,10.0718,0.0000,23.8788,18.8460,25.9044,23.0771,29.1807,22.9151,30.1146,31.9300,17.6784,27.0888,9.9481,27.5004,11.9635,10.3362
3,25.4014,21.0302,26.4754,0.0000,12.1517,5.4567,5.2835,19.9805,16.1660,20.9815,22.4687,10.7027,6.6411,16.7480,7.0527,18.3522,25.4014
4,19.2797,14.9085,20.3537,6.6943,0.0000,5.8522,9.7068,14.8417,9.0686,15.9114,17.9854,8.4156,9.6451,10.6264,7.9013,13.0229,19.2797
5,26.1492,21.7780,27.2232,4.1030,8.5291,0.0000,7.1155,16.4265,12.6121,17.4276,18.9148,9.7848,4.2234,17.4958,4.6350,19.1000,26.1492
6,21.5524,17.4290,24.0462,4.0533,10.1076,7.1273,0.0000,21.6511,17.8366,22.6521,24.1393,5.4057,8.3117,16.3528,8.7233,20.0228,21.5524
7,29.8791,25.5079,32.7576,18.6105,14.8266,16.4680,21.6230,0.0000,6.0290,1.0704,4.2170,21.2254,17.4946,20.3755,15.1319,19.1937,29.8791
8,23.3786,19.0075,26.2571,14.8743,9.0896,12.7317,17.8868,5.8346,0.0000,6.9044,9.6782,15.4883,13.7583,13.8751,12.0146,12.6933,23.3786
9,30.8130,26.4418,33.6915,19.6116,15.8963,17.4690,22.6241,1.0704,7.0987,0.0000,3.3186,22.2951,18.4956,21.3094,16.5027,20.1276,30.8130


In [140]:
time_matrix

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0.000000,0.154806,0.209361,0.529694,0.463472,0.588278,0.515389,0.620111,0.517917,0.642194,0.661500,0.399556,0.563667,0.268333,0.584389,0.316222,0.000000
1,0.122306,0.000000,0.197694,0.410389,0.344167,0.469000,0.396083,0.519944,0.417778,0.542056,0.561361,0.280278,0.444361,0.168611,0.465083,0.246111,0.122306
2,0.253139,0.234194,0.000000,0.504806,0.438611,0.563417,0.516306,0.577000,0.474833,0.599111,0.618417,0.400472,0.538806,0.235167,0.559500,0.257944,0.253139
3,0.532444,0.413694,0.566056,0.000000,0.238222,0.133444,0.121333,0.393028,0.294000,0.411222,0.440083,0.191361,0.108806,0.336028,0.129528,0.393111,0.532444
4,0.442500,0.323750,0.476083,0.180778,0.000000,0.183889,0.248500,0.303194,0.185306,0.328000,0.365361,0.199528,0.185333,0.246056,0.170472,0.302361,0.442500
5,0.568833,0.450111,0.602444,0.108444,0.202972,0.000000,0.176167,0.350361,0.251333,0.368556,0.397417,0.231583,0.095611,0.372417,0.116333,0.429500,0.568833
6,0.497583,0.389722,0.555778,0.102194,0.253222,0.178639,0.000000,0.438222,0.339194,0.456444,0.485278,0.112361,0.154028,0.368389,0.174750,0.438333,0.497583
7,0.601806,0.483083,0.526361,0.365194,0.300639,0.357444,0.432917,0.000000,0.121361,0.024833,0.089583,0.452306,0.333139,0.391722,0.314639,0.376056,0.601806
8,0.513833,0.395111,0.438389,0.267083,0.185583,0.259333,0.334806,0.117972,0.000000,0.142778,0.196528,0.337250,0.235056,0.303750,0.220194,0.288083,0.513833
9,0.623917,0.505167,0.548472,0.383389,0.325472,0.375639,0.451111,0.024833,0.146167,0.000000,0.069056,0.477111,0.351361,0.413833,0.331444,0.398139,0.623917


In [141]:
distance_matrix.to_csv('distance_matrix.csv', index=False)

print("DataFrame saved to 'distance_matrix.csv'")

DataFrame saved to 'distance_matrix.csv'


In [142]:
time_matrix.to_csv('time_matrix.csv', index=False)

print("DataFrame saved to 'time_matrix.csv'")

DataFrame saved to 'time_matrix.csv'


#### EV : Volvo C40 Recharge
- Mileage : 530 km/full charge
- Battery capacity : 78 kWh
- Charging time : 27 min
- Power : 300 kW

In [143]:
mileage = 530 #km/full charge
battery_capacity = 78 #kWh
battery_expenditure_per_km = battery_capacity/mileage

In [144]:
battery_expenditure_matrix = np.zeros((number_of_node, number_of_node))
battery_expenditure_matrix = pd.DataFrame(battery_expenditure_matrix)

for i in range(number_of_node):
    for j in range(number_of_node):
            battery_expenditure_matrix.iloc[i,j]=distance_matrix.iloc[i,j]*battery_expenditure_per_km

In [145]:
battery_expenditure_matrix

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0.000000,0.924977,1.311121,3.654447,2.913756,3.952540,3.391087,4.426338,3.504231,4.563780,4.830952,2.596561,4.126862,1.653585,4.187423,1.872633,0.000000
1,0.651182,0.000000,1.391976,2.970770,2.230079,3.268862,2.707409,3.785134,2.863027,3.922561,4.189733,1.912869,3.443170,1.102346,3.503745,1.475745,0.651182
2,1.521177,1.482265,0.000000,3.514238,2.773562,3.812346,3.396252,4.294518,3.372411,4.431960,4.699132,2.601727,3.986654,1.464060,4.047229,1.760666,1.521177
3,3.738319,3.095011,3.896380,0.000000,1.788363,0.803062,0.777572,2.940526,2.379147,3.087843,3.306714,1.575114,0.977369,2.464800,1.037945,2.700890,3.738319
4,2.837390,2.194081,2.995450,0.985199,0.000000,0.861267,1.428548,2.184250,1.334624,2.341678,2.646908,1.238522,1.419468,1.563885,1.162833,1.916578,2.837390
5,3.848373,3.205064,4.006433,0.603838,1.255226,0.000000,1.047187,2.417485,1.856120,2.564817,2.783688,1.440027,0.621557,2.574854,0.682132,2.810943,3.848373
6,3.171863,2.565023,3.538875,0.596523,1.487534,1.048923,0.000000,3.186388,2.625009,3.333705,3.552576,0.795556,1.223231,2.406638,1.283806,2.946752,3.171863
7,4.397302,3.753993,4.820930,2.738904,2.182028,2.423592,3.182253,0.000000,0.887287,0.157531,0.620615,3.123738,2.574677,2.998658,2.226959,2.824733,4.397302
8,3.440624,2.797330,3.864252,2.189048,1.337715,1.873722,2.632397,0.858677,0.000000,1.016119,1.424339,2.279410,2.024806,2.041996,1.768186,1.868071,3.440624
9,4.534743,3.891435,4.958372,2.886235,2.339455,2.570909,3.329585,0.157531,1.044714,0.000000,0.488398,3.281166,2.721994,3.136100,2.428699,2.962175,4.534743


In [146]:
battery_expenditure_matrix.to_csv('battery_expenditure_matrix.csv', index=False)

print("DataFrame saved to 'battery_expenditure_matrix.csv'")

DataFrame saved to 'battery_expenditure_matrix.csv'


In [147]:
save_nodes=pd.DataFrame(nodes)
save_nodes.to_csv('nodes.csv', index=False)

print("DataFrame saved to 'nodes.csv'")

DataFrame saved to 'nodes.csv'
